In [ ]:
# imports
# type: ignore
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime as dt
import cProfile
from pprint import pprint
from typing import Any

import aiohttp

from tests.mock_wynnapi import MockWynnApi

In [ ]:
def className(obj: object) -> str:
    return obj.__class__.__name__

def getIndent(depth: int) -> str:
    return '    ' * depth

def printDictTypes(dict_: dict[str, Any], depth: int = 0) -> None:
    for key, value in dict_.items():
        if isinstance(value, dict):
            print(f'{getIndent(depth)}{key}: dict')
            printDictTypes(value, depth + 1)
        else:
            print(f'{getIndent(depth)}{key}: {className(value)}')

async def getFullPlayerStats(username_or_uuid: str) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/player/{username_or_uuid}?fullResult") as resp:
            return (await resp.json(), dict(resp.headers))

async def getGuildStats(guildname_or_prefix: str, is_prefix: bool = False) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/guild/{'prefix/' if is_prefix else ''}{guildname_or_prefix}") as resp:
            return (await resp.json(), dict(resp.headers))

async def getOnlinePlayers() -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.wynncraft.com/v3/player?identifier=uuid") as resp:
            return (await resp.json(), dict(resp.headers))

# Sandbox

In [2]:
from loguru import logger

from kans import config
from kans.api import WynnApi
from kans.db import KansDatabase


api = WynnApi(logger)
db = KansDatabase(config, logger)


In [5]:
# pyright: reportPrivateUsage=none, reportUnusedVariable=false
from datetime import datetime
from uuid import UUID
from kans.db.database_query import DatabaseQuery

from kans.db.model.character_history.character_history import CharacterHistory
from kans.db.model.character_history.character_history_id import CharacterHistoryId
from kans.db.repository.character_history_repository import CharacterHistoryRepository
from kans.util.api_response_adapter import ApiResponseAdapter
from kans.util.db_model_dict_adapter import DbModelDictAdapter
from kans.util.db_model_id_dict_adapter import DbModelIdDictAdapter
from tests.fixtures_api import FixturesApi


_adapter = ApiResponseAdapter()
_db = DatabaseQuery(config['DB_USERNAME'], config['DB_PASSWORD'], config['SCHEMA_NAME'])
_repo = CharacterHistoryRepository(_db, DbModelDictAdapter(), DbModelIdDictAdapter())
_repo._TABLE_NAME = "test_character_history"
await _repo.create_table()

fixtures = FixturesApi()
raw_test_data = [
        entity
        for datum in fixtures.get_players()[:10]
        for entity in _adapter.Player.to_character_history(datum)
]
raw_test_data = raw_test_data[:10]  # Get 10

now = datetime.now()
test_data: list[CharacterHistory] = []
for i, entity in enumerate(raw_test_data):
    as_dict = _repo._adapt(entity)  # Change entity to dict
    as_dict["character_uuid"] = UUID(int=i).bytes  # Modify the dict
    as_dict["datetime"] = now
    test_data.append(entity.__class__(**as_dict))  # Change modified dict to entity

n = await _repo.insert(test_data)  # Insert modified entity

found: list[CharacterHistory] = []
for entity in test_data:
    # Find the inserted entity
    res = await _repo.find_one(CharacterHistoryId(entity.character_uuid, entity.datetime))
    if res:
        found.append(res)

found_uuids = {entity.character_uuid.uuid for entity in found}
test_uuids = {entity.character_uuid.uuid for entity in test_data}

c:\Users\user\Documents\code\repositories\dev\kans\.venv\Lib\site-packages\aiomysql\cursors.py:458: Warning: UNSIGNED for decimal and floating point data types is deprecated and support for it will be removed in a future release.
  await self._do_get_result()
c:\Users\user\Documents\code\repositories\dev\kans\.venv\Lib\site-packages\aiomysql\cursors.py:458: Warning: Table 'test_character_history' already exists
  await self._do_get_result()


In [3]:
from uuid import UUID


for i in range(10):
    print(UUID(int=i))

00000000-0000-0000-0000-000000000000
00000000-0000-0000-0000-000000000001
00000000-0000-0000-0000-000000000002
00000000-0000-0000-0000-000000000003
00000000-0000-0000-0000-000000000004
00000000-0000-0000-0000-000000000005
00000000-0000-0000-0000-000000000006
00000000-0000-0000-0000-000000000007
00000000-0000-0000-0000-000000000008
00000000-0000-0000-0000-000000000009


# Archive

In [ ]:
# dict-mapping vs match-case
# from timeit import timeit


# def perf_dict(inpt: str):
#     mapping = {
#         "foo": 1,
#         "bar": 2,
#         "baz": 3,
#         "qux": 4,
#         "quux": 5,
#     }
#     return mapping[inpt]

# def perf_matchcase(inpt: str):
#     match inpt:
#         case "foo":
#             return 1
#         case "bar":
#             return 2
#         case "baz":
#             return 3
#         case "qux":
#             return 4
#         case "quux":
#             return 5

# print(timeit('perf_dict("qux")', globals=globals(), number=10_000_000))
# print(timeit('perf_matchcase("qux")', globals=globals(), number=10_000_000))

In [ ]:
# wynnrepo init
# repo = WynnDataDatabase(config, logger)
# mockdata = MockWynnApi()
# p: PlayersResponse = mockdata.onlineuuids  # type: ignore
# ps: list[PlayerResponse] = mockdata.onlineplayerstats  # type: ignore
# gs: list[GuildResponse] = mockdata.onlineguildstats  # type: ignore

In [ ]:
# wynnrepo createtable
# db = WynnDataDatabase(config, None)
# await db.character_history_repository.create_table()
# await db.character_info_repository.create_table()
# await db.guild_history_repository.create_table()
# await db.guild_info_repository.create_table()
# await db.guild_member_history_repository.create_table()
# await db.kans_uptime_repository.create_table()
# await db.online_players_repository.create_table()
# await db.player_activity_history_repository.create_table()
# await db.player_history_repository.create_table()
# await db.player_info_repository.create_table()

In [ ]:
# wynnrepo insert
# print(await repo.character_history_repository.insert(CharacterHistory.from_responses(ps)))
# print(await repo.character_info_repository.insert(CharacterInfo.from_responses(ps)))
# print(await repo.guild_history_repository.insert(GuildHistory.from_responses(gs)))
# print(await repo.guild_info_repository.insert(GuildInfo.from_responses(gs)))
# print(await repo.guild_member_history_repository.insert(GuildMemberHistory.from_responses(gs)))
# print(await repo.online_players_repository.insert(OnlinePlayers.from_response(p)))
# # print(await repo.player_activity_history_repository.insert())
# print(await repo.player_history_repository.insert(PlayerHistory.from_responses(ps)))
# print(await repo.player_info_repository.insert(PlayerInfo.from_responses(ps)))

In [ ]:
# wynnrepo count
# print(await repo.character_history_repository.count())
# print(await repo.character_info_repository.count())
# print(await repo.guild_history_repository.count())
# print(await repo.guild_info_repository.count())
# print(await repo.guild_member_history_repository.count())
# print(await repo.online_players_repository.count())
# # print(await repo.player_activity_history_repository.count())
# print(await repo.player_history_repository.count())
# print(await repo.player_info_repository.count())

In [ ]:
# list of objects: `in`, `min()`, `max()`
# class Foo:
#     def __init__(self, weight: int, value: str):
#         self.weight = weight
#         self.value = value

#     def __eq__(self, value: object) -> bool:
#         return self.value == value

#     def __lt__(self, other: Foo) -> bool:
#         return self.weight < other.weight

# ch = 'abcdefghijklmnopqrstuvwxyz'
# foos = []
# for i in range(26):
#     foos.append(Foo(i, ch[i]))

# # 'value' in 'iterable' checks for equality of 'value' with each element in 'iterable'
# print('g' in foos)
# # min and max use the __lt__ magic method to compare elements in 'iterable'
# print(min(foos).value)
# print(max(foos).value)

In [ ]:
# dbmodel creation algorithm perfomance compariosn
# from tests.mock_wynnapi import MockWynnApi
# from vindicator.api.wynn.player_response import PlayerResponse
# from vindicator.db.wynndata.model.character_history.character_history import CharacterHistory
# from vindicator.db.wynndata.model.character_info.character_info import CharacterInfo
# from vindicator.db.wynndata.model.player_history.player_history import PlayerHistory
# from vindicator.db.wynndata.model.player_info.player_info import PlayerInfo

# mockwynnapi = MockWynnApi()
# data: list[PlayerResponse] = mockwynnapi.onlineplayerstats  # type: ignore

# def withloop():
#     playerinfo = []
#     characterinfo = []
#     playerhistory = []
#     characterhistory = []
#     for resp in data:
#         playerinfo.extend(PlayerInfo.from_responses((resp,)))
#         characterinfo.extend(CharacterInfo.from_responses((resp,)))
#         playerhistory.extend(PlayerHistory.from_responses((resp,)))
#         characterhistory.extend(CharacterHistory.from_responses((resp,)))

# def without():
#     playerinfo: tuple[PlayerInfo, ...] = PlayerInfo.from_responses(data)
#     characterinfo: tuple[CharacterInfo, ...] = CharacterInfo.from_responses(data)
#     playerhistory: tuple[PlayerHistory, ...] = PlayerHistory.from_responses(data)
#     characterhistory: tuple[CharacterHistory, ...] = CharacterHistory.from_responses(data)

# cProfile.run('withloop()', sort='time')
# cProfile.run('without()', sort='time')

In [ ]:
# += vs extend()
# r = range(50_000_000)
# def add():
#     l = []
#     l += tuple(r)
# def extend():
#     l = []
#     l.extend(r)

# cProfile.run('add()')
# cProfile.run('extend()')

In [ ]:
# moto-bot repository structure
# from __future__ import annotations
# ID = TypeVar('ID', contravariant=True)
# T = TypeVar('T')
# S = TypeVar('S')
# U = TypeVar('U', bound=type)

# def interface(t: U) -> U: return t
# def abstract(t: U) -> U: return t

# @interface
# class Table(Generic[T, ID], Protocol):
#     def create(self, entity: T) -> bool: ...
#     def exists(self, id_: ID) -> bool: ...
#     def count(self) -> float: ...
#     def findOne(self, id_: ID) -> None | T: ...
#     def findAll(self) -> None | list[T]: ...
#     def update(self, entity: T) -> bool: ...
#     def delete(self, id_: ID) -> bool: ...

# @abstract
# class MariaRepository(ABC, Generic[S]):
#     def __init__(self, db: Any, logger: Any) -> None:
#         self._db: Any = db
#         self._logger: Any = logger
#     def _prepareStatement(self, connection: Any, sql: str, objects: list[object]) -> Any: ...
#     def _execute(self, sql: str, strings: list[object]) -> bool: ...
#     # def _execute(self, connection: Any, sql: str, objects: list[object]) -> bool: ...
#     def _executeQuery(self, sql: str, objects: list[object]) -> None | Any: ...
#     def _logResponseException(self, e: Any) -> None: ...
#     def _bindAll(self, res: Any) -> list[S]: ...
#     @abstractmethod
#     def _bind(self, res: Any) -> S: ...

# @interface
# class GuildId(Protocol):
#     def getName(self) -> str: ...

# class Guild(GuildId):
#     def __init__(self, name: str, prefix: str, createdAt: dt) -> None:
#         self._name: str = name
#         self._prefix: str = prefix
#         self._createdAt: dt = createdAt
#     @override
#     def getName(self) -> str: return self._name
#     def getPrefix(self) -> str: return self._prefix
#     def getCreatedAt(self) -> dt: return self._createdAt

# @interface
# class CharacterHistoryBase(Table[Guild, GuildId], Protocol):
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...

# class CharacterHistoryTable(MariaRepository[Guild], CharacterHistoryBase):
#     """class::MariaGuildRepository"""
#     _DB_FORMAT: Any
#     def __init__(self, db: Any, logger: Any) -> None:
#         super().__init__(db, logger)
#     @override
#     def _bind(self, res: Any) -> Guild: ...
#     @override
#     def create(self, entity: Guild) -> bool: ...
#     @override
#     def exists(self, id_: GuildId) -> bool: ...
#     @override
#     def count(self) -> float: ...
#     @override
#     def findOne(self, id_: GuildId) -> None | Guild: ...
#     @override
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...
#     @override
#     def findAll(self) -> None | list[Guild]: ...
#     @override
#     def update(self, entity: Guild) -> bool: ...
#     @override
#     def delete(self, id_: GuildId) -> bool: ...
